In [1]:
import torch 
from torch import nn
from torch.nn import functional as F

## neuron->layer->module(block)->model

每个layer/module必须提供：

- 将输入数据作为其前向传播函数的参数。
- 通过前向传播函数来生成输出。
- 计算其输出关于输入的梯度，可通过其反向传播函数进行访问。
- 存储和访问前向传播计算所需的参数。
- 根据需要初始化模型参数。

如果我们组合已有的nn提供的layer，我们只需要实现前向传播

layer/module一般实现成nn.Module的子类。对于一个nn.Module对象net，我们只需要对它调用net(X)，相当于执行了forward函数

In [2]:
#一个自定义layer
class FixedHiddenMLP(nn.Module):
    def __init__(self):
        super().__init__()
        # 不计算梯度的随机权重参数。因此其在训练期间保持不变
        self.rand_weight = torch.rand((20, 20), requires_grad=False)
        self.linear = nn.Linear(20, 20)

    def forward(self, X):
        X = self.linear(X)
        # 使用创建的常量参数以及relu和mm函数
        X = F.relu(torch.mm(X, self.rand_weight) + 1)
        # 复用全连接层。这相当于两个全连接层共享参数
        X = self.linear(X)
        # 控制流
        while X.abs().sum() > 1:
            X /= 2
        return X.sum()

In [3]:
class MLP(nn.Module):
    # 用模型参数声明层。这里，我们声明两个全连接的层
    def __init__(self):
        # 调用MLP的父类Module的构造函数来执行必要的初始化。
        # 这样，在类实例化时也可以指定其他函数参数，例如模型参数params（稍后将介绍）
        super().__init__()
        self.hidden = nn.Linear(20, 256)  # 隐藏层
        self.out = nn.Linear(256, 10)  # 输出层

    # 定义模型的前向传播，即如何根据输入X返回所需的模型输出
    def forward(self, X):
        # 注意，这里我们使用ReLU的函数版本，其在nn.functional模块中定义。
        return self.out(F.relu(self.hidden(X)))

nn.Sequential可以连接layers，也可以连接modules组成更大的modules，返回一个nn.Module类

In [4]:
#Sequential作为一个子module
class NestMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(nn.Linear(20, 64), nn.ReLU(),
                                 nn.Linear(64, 32), nn.ReLU())
        self.linear = nn.Linear(32, 16)

    def forward(self, X):
        return self.linear(self.net(X))

In [5]:
X = torch.rand(2, 20)
chimera = nn.Sequential(NestMLP(), nn.Linear(16, 20), FixedHiddenMLP())
chimera(X)

tensor(0.0869, grad_fn=<SumBackward0>)

## 参数访问
- ``state_dict``方法可以访问nn.Module类的所有参数
- ``parameter()``方法会生成一个访问nn.Module类所有参数的迭代器
- 当通过Sequential类定义模型时， 我们可以通过索引来访问模型的任意层

In [6]:
print(chimera[1].state_dict())

OrderedDict([('weight', tensor([[-0.0292,  0.0929, -0.2302, -0.0729,  0.1692, -0.1976,  0.0834, -0.1896,
         -0.0728, -0.2421,  0.0893,  0.1170,  0.0648,  0.0758, -0.1985,  0.1306],
        [ 0.1271, -0.0399, -0.0378,  0.0349, -0.0319, -0.2491,  0.1688, -0.2114,
          0.0871,  0.0473, -0.0180,  0.0832,  0.2257,  0.0589,  0.2080,  0.1223],
        [ 0.2117, -0.1851,  0.1861,  0.1620, -0.0733,  0.2227,  0.0680, -0.2488,
         -0.1128,  0.0395,  0.0548, -0.1617, -0.0193,  0.1155,  0.0379,  0.1138],
        [-0.0965,  0.2233,  0.1100, -0.0832,  0.1523,  0.0557,  0.1497,  0.1412,
         -0.0956,  0.2037, -0.0574,  0.0406, -0.1458, -0.1391,  0.0863, -0.2172],
        [-0.1007, -0.2260,  0.1123, -0.2036, -0.1287, -0.1272, -0.0531, -0.0556,
          0.1041,  0.1706, -0.1618, -0.1017, -0.1912, -0.0980,  0.1026, -0.0382],
        [ 0.0472, -0.1707,  0.1211, -0.1312,  0.1688, -0.0168,  0.2379, -0.1413,
          0.2223,  0.2262, -0.2076, -0.0488, -0.1225, -0.1564,  0.0976,  0.0236]

In [7]:
for i in chimera.parameters():
    print (i.requires_grad,i.shape)

True torch.Size([64, 20])
True torch.Size([64])
True torch.Size([32, 64])
True torch.Size([32])
True torch.Size([16, 32])
True torch.Size([16])
True torch.Size([20, 16])
True torch.Size([20])
True torch.Size([20, 20])
True torch.Size([20])


In [14]:
print(type(chimera[1].bias))
print(chimera[1].bias)
print(chimera[1].bias.data)
#对于自定义层，我们直接访问数据成员
print(chimera[2].linear.bias.data)

<class 'torch.nn.parameter.Parameter'>
Parameter containing:
tensor([-0.0172,  0.1755, -0.0482,  0.1067, -0.0745,  0.1671, -0.2355, -0.2153,
        -0.0984, -0.1302, -0.0109, -0.0282, -0.1354, -0.2063, -0.1290,  0.0137,
         0.1248,  0.2119,  0.1369,  0.2051], requires_grad=True)
tensor([-0.0172,  0.1755, -0.0482,  0.1067, -0.0745,  0.1671, -0.2355, -0.2153,
        -0.0984, -0.1302, -0.0109, -0.0282, -0.1354, -0.2063, -0.1290,  0.0137,
         0.1248,  0.2119,  0.1369,  0.2051])
tensor([-0.0439,  0.0461, -0.1094, -0.0604,  0.0629, -0.1719,  0.0335, -0.0089,
         0.1701,  0.0710,  0.1843,  0.1261,  0.0046,  0.2213,  0.0181,  0.1970,
         0.0012,  0.1993,  0.0825, -0.2197])


## 自定义初始化
这里``apply``方法会深度优先遍历每一个模块(module)

In [15]:
def my_init(m):
    if type(m) == nn.Linear:
        print("Init", *[(name, param.shape)
                        for name, param in m.named_parameters()][0])
        nn.init.uniform_(m.weight, -10, 10)
        m.weight.data *= m.weight.data.abs() >= 5

chimera.apply(my_init)
chimera[1].weight[:2]

Init weight torch.Size([64, 20])
Init weight torch.Size([32, 64])
Init weight torch.Size([16, 32])
Init weight torch.Size([20, 16])
Init weight torch.Size([20, 20])


tensor([[ 0.0000, -8.8114,  5.6600, -0.0000, -9.6173, -0.0000, -0.0000,  7.6698,
          6.4698,  0.0000, -9.2073, -0.0000, -0.0000,  0.0000,  0.0000,  5.9374],
        [ 0.0000, -0.0000, -5.3784,  0.0000,  0.0000, -9.8710, -0.0000, -0.0000,
         -0.0000, -6.8422,  0.0000, -6.8476, -6.7283, -6.1795, -5.9927, -8.6711]],
       grad_fn=<SliceBackward0>)

## 参数共享


In [18]:
# 我们需要给共享层一个名称，以便可以引用它的参数
shared = nn.Linear(8, 8)
net = nn.Sequential(nn.Linear(4, 8), nn.ReLU(),
                    shared, nn.ReLU(),
                    shared, nn.ReLU(),
                    nn.Linear(8, 1))
Xx=torch.rand(4)
net(Xx)
# 检查参数是否相同
print(net[2].weight.data[0] == net[4].weight.data[0])
net[2].weight.data[0, 0] = 100
# 确保它们实际上是同一个对象，而不只是有相同的值
print(net[2].weight.data[0] == net[4].weight.data[0])

tensor([True, True, True, True, True, True, True, True])
tensor([True, True, True, True, True, True, True, True])


## 延后初始化 lazy initialization


In [ ]:

net = nn.Sequential(nn.LazyLinear(256), nn.ReLU(), nn.LazyLinear(10))
print(net[0].weight)  # 尚未初始化
X = torch.rand(2, 20)
net(X)
print(net[0].weight)

<UninitializedParameter>
Parameter containing:
tensor([[-0.1950, -0.1744, -0.0641,  ..., -0.0823, -0.1154, -0.0948],
        [-0.0350,  0.0605, -0.1897,  ...,  0.0568, -0.0886,  0.0389],
        [ 0.1849, -0.1263,  0.0249,  ...,  0.2083, -0.0185, -0.1320],
        ...,
        [-0.0455,  0.1780, -0.1234,  ...,  0.1639, -0.1975, -0.1750],
        [ 0.1486,  0.1205,  0.1950,  ..., -0.0156, -0.0652,  0.1930],
        [-0.0490, -0.0500,  0.1785,  ...,  0.0401, -0.1717, -0.0313]],
       requires_grad=True)


## 读写文件

In [8]:
import os
print(os.getcwd())

d:\a_university\1.3MLDL\LMDL


In [9]:
x=torch.arange(4)
torch.save(x,'test_for_saving')

In [10]:
x2=torch.load('test_for_saving')
x2

tensor([0, 1, 2, 3])

In [11]:
y = torch.zeros(4)
torch.save([x, y],'x-files')
x2, y2 = torch.load('x-files')
(x2, y2)

(tensor([0, 1, 2, 3]), tensor([0., 0., 0., 0.]))

In [12]:
class MLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.hidden = nn.Linear(20, 256)
        self.output = nn.Linear(256, 10)

    def forward(self, x):
        return self.output(F.relu(self.hidden(x)))

net = MLP()
X = torch.randn(size=(2, 20))
Y = net(X)
torch.save(net.state_dict(), 'mlp.params')

加载参数用``load_state_dict``方法

In [13]:
clone = MLP()
clone.load_state_dict(torch.load('mlp.params'))
clone.eval()
Y_clone = clone(X)
Y_clone == Y

tensor([[True, True, True, True, True, True, True, True, True, True],
        [True, True, True, True, True, True, True, True, True, True]])